In [2]:

import numpy as np

from time import sleep

import googlemaps
import folium
import pandas as pd
import getpass
from pymongo import MongoClient
from dotenv import load_dotenv
from bson import json_util
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [3]:
load_dotenv
maps_token = getpass.getpass("maps_token: ")
map_client = googlemaps.Client(maps_token)

In [4]:
def extracting_data(business,keyword, radius):
    sleep(1)
    location = (41.4036522,2.189654)  
    radius = radius # radius in meters
    business = business
    keyword = keyword
    results = []
    response = map_client.places_nearby(location=location, type=business, radius=radius, keyword=keyword)
    results.extend(response['results'])
    while True:
        try:
            sleep(2)
            response = map_client.places_nearby(location=location, type=business, radius=radius, keyword=keyword, page_token=next_page)
            results.extend(response['results'])
            next_page = response['next_page_token']
        except:
            break
    return results    

In [5]:
def dataframe(data, business, keyword, color, icon_color, icon):
    list_local = []
    df = pd.DataFrame(columns = ['Category','Type', 'Name', 'Address', 'Latitude', 'Longitude', 'Icon', 'Icon Color', 'Color', 'Prefix'])
    for result in data:
        local = {}
        local['Category'] = business
        local['Type'] = keyword
        local['Color'] = color
        local['Icon Color'] = icon_color
        local['Icon'] = icon
        local['Prefix'] = 'fa'
        local['Name'] = result['name']
        local['Address'] = result['vicinity']
        local['Latitude'] = result['geometry']['location']['lat']
        local['Longitude'] = result['geometry']['location']['lng']
        list_local.append(local)
    df = pd.DataFrame(list_local)
    return df    

In [6]:
bars = extracting_data('Bar', 'Bar', 1500)

In [7]:
restaurant = extracting_data('Restaurant', 'Restaurant', 1000)

In [8]:
school = extracting_data('School', 'public School', 2000)

In [9]:
veg_restaurant = extracting_data('Restaurant', 'vegetarian', 500)

In [10]:
hairdresser = extracting_data('hairdresser', 'hairdresser', 300)

In [11]:
tech_startups = extracting_data('tech startups', 'startups', 2000)

In [12]:
coffee = extracting_data('Coffe', 'Coffee Shop', 500)

In [13]:
metro = extracting_data('Metro', 'Metro Stations', 700)

In [14]:
bus = extracting_data('Bus', 'Bus Station', 700)

In [15]:
travel_agency = extracting_data('Travel Agency', 'Travel Agency', 500)

In [16]:
df_bars = dataframe(bars,'Bar', 'Bar', 'red', 'white', 'beer-mug-empty' )

In [17]:
df_restaurants = dataframe(restaurant,'Restaurant', 'Restaurant', 'orange', 'white', 'utensils')

In [18]:
df_school = dataframe(school, 'School', 'Public School', 'blue', 'black', 'graduation-cap')

In [19]:
df_veg = dataframe(veg_restaurant,'Restaurant', 'Vegetarian', 'green', 'white', 'wheat-awn')

In [20]:
df_hairdresser = dataframe(hairdresser, 'Hairdresser', 'Health & Beauty', 'white', 'black', 'scissors')

In [21]:
df_tech_startups = dataframe(tech_startups, 'Tech startups', 'Startups', 'black', 'white', 'desktop')

In [22]:
df_coffee = dataframe(coffee, 'Coffee', 'Coffee  Shop', 'beige', 'back', 'mug-hot')

In [23]:
df_metro = dataframe(metro, 'Metro', 'Metro Station', 'darkred', 'white', 'train-subway')

In [24]:
df_bus = dataframe(bus, 'Bus', 'Bus Station', 'darkblue', 'white', 'bus')

In [25]:
df_travel_agency = dataframe(travel_agency, 'Travel Agency', 'Travel Agency', 'purple', 'black', 'plane')

In [26]:
basketball = {'Category': 'Sports','Type': 'basketball', 'Name': 'Federació Catalana de Basquetbol', 'Address': 'C. de Guipúscoa, 27,Barcelona', 'Latitude':'41.4144622', 'Longitude':'2.1941339', 'Icon':'basketball', 'Icon Color':'black', 'Color':'lightblue', 'Prefix':'fa'}
index =['0']
df_basketball = pd.DataFrame(basketball, index = index)

In [27]:
company = {'Category': 'Business','Type': 'Tech', 'Name': 'Lessa entertainment', 'Address': 'Av. Diagonal, 211, Barcelona', 'Latitude':'41.4039759', 'Longitude':'2.1906131', 'Icon':'building', 'Icon Color':'white', 'Color':'black', 'Prefix':'fa'}
df_company = pd.DataFrame(company, index = index)

# Exporting the DataFrame to Mongodb

In [28]:
def export_to_mongodb(data):
    client = MongoClient("localhost", 27017)
    db = client["Project-Spot"]
    for i, v in enumerate(data):
        file_csv = f"Project-Spot_{i}.csv"
        v.to_csv(f'data/{file_csv}', index=False)
        collection_name = v['Type'].iloc[0]
        collection = db[collection_name]
        result = v.to_dict(orient='records')
        collection.insert_many(result)
    

# Create the Map layer and add it to the FeatureGroup

In [29]:
city_map = [
    df_coffee, df_tech_startups,
    df_hairdresser, df_veg, 
    df_school, df_bars,
    df_restaurants, df_basketball,
    df_company, df_metro,
    df_bus, df_travel_agency
]

layers_name = [
    'Coffee Shop', 'Startups',
    'Health & Beauty', 'Vegetarian',
    'Public School', 'Bar',
    'Restaurant', 'basketball',
    'Business', 'Metro Station',
    'Bus Station', 'Travel Agency'
]

In [30]:
export_to_mongodb(city_map)

# Creating the MAP

In [31]:
def add_spot(data, layer):
    main_map = folium.Map(location = ['41.4034532','2.1896495'],tiles="Cartodb dark_matter", zoom_start=15)
    layer_list = []
    red_cicle = folium.FeatureGroup(name = 'Red Cicle')    
    folium.Circle(location=['41.4034532', '2.1896495'], radius=2500, color='red', fill=True).add_to(red_cicle)
    layer_list.append(red_cicle)
    yellow_cicle = folium.FeatureGroup(name = 'Yellow Cicle')    
    folium.Circle(location=['41.4034532', '2.1896495'], radius=1500, color='yellow', fill=True).add_to(yellow_cicle)
    layer_list.append(yellow_cicle)
    green_cicle = folium.FeatureGroup(name = 'Green Cicle')    
    folium.Circle(location=['41.4034532', '2.1896495'], radius=700, color='green', fill=True).add_to(green_cicle)
    layer_list.append(green_cicle)
    for df, lay in zip(data, layer):
        layer = folium.FeatureGroup(name = f'{lay}')
        for k, v in df.iterrows():
            icon = folium.Icon(
                color= v['Color'], 
                icon_color= v['Icon Color'], 
                icon= v['Icon'], prefix= v['Prefix']
                )
            folium.Marker(location=[v['Latitude'], v['Longitude']], popup= v['Name'], icon = icon).add_to(layer)
        layer_list.append(layer)
    for v in layer_list:
        v.add_to(main_map)
    folium.LayerControl().add_to(main_map)
    html_file = 'barcelona_map.html'
    main_map.save(f'html/{html_file}')
    return main_map

In [32]:
add_spot(city_map, layers_name)